In [1]:
import sys
sys.path.insert(0, 'src/')
from utils import *
from template_attacks import *
from traces_simulator import *
from metrics import *
from scoop import Scoop
from custom_cnn import CNNModel
from custom_mlp import MLPModel

import os
import h5py
import matplotlib.pyplot as plt
from tqdm import tqdm, trange
import torch.nn.functional as F
import random
import optuna
from load_ascad_data import load_ascad_data
from train_model import train_model_h, train_model_classic
import torch
import numpy as np

seed = 42
torch.manual_seed(seed)
np.random.seed(seed)
random.seed(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
torch.cuda.manual_seed(seed)
os.environ['PYTHONHASHSEED']=str(seed)
#torch.use_deterministic_algorithms(True)

print('__Python VERSION:', sys.version)
print('__pyTorch VERSION:', torch.__version__)
print('__CUDA VERSION')
from subprocess import call
# call(["nvcc", "--version"]) does not work
! nvcc --version
print('__CUDNN VERSION:', torch.backends.cudnn.version())
print('__Number CUDA Devices:', torch.cuda.device_count())
print('__Devices')
call(["nvidia-smi", "--format=csv", "--query-gpu=index,name,driver_version,memory.total,memory.used,memory.free"])
print('Active CUDA Device: GPU', torch.cuda.current_device())
print ('Available devices ', torch.cuda.device_count())
print ('Current cuda device', torch.cuda.current_device())
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

__Python VERSION: 3.11.10 (main, Oct  3 2024, 07:29:13) [GCC 11.2.0]
__pyTorch VERSION: 2.5.1
__CUDA VERSION
/bin/bash: ligne 1: nvcc : commande introuvable


/home/nrousselot/miniconda3/envs/curse_of_masking/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


__CUDNN VERSION: 90100
__Number CUDA Devices: 1
__Devices
index, name, driver_version, memory.total [MiB], memory.used [MiB], memory.free [MiB]
0, NVIDIA RTX 4500 Ada Generation, 535.183.01, 24570 MiB, 7 MiB, 24216 MiB
Active CUDA Device: GPU 0
Available devices  1
Current cuda device 0


In [2]:
def objective(trial):
    n_classes = 256
    signal_length = 700
    lr = trial.suggest_float('lr', 1e-4, 1e-2, log=True)
    beta1 = trial.suggest_float('beta1', 0.9, 0.999, step=0.001)
    beta2 = trial.suggest_float('beta2', 0.9, 0.999, step=0.001)
    n_epochs = 60
    activation = trial.suggest_categorical('activation', ['ReLU', 'SeLU', 'ELU', 'Tanh'])
    n_linear = trial.suggest_int('n_linear', 2, 8)
    linear_size = trial.suggest_categorical('linear_size', [256])
    input_bn = trial.suggest_categorical('input_bn', [True, False])
    dense_bn = trial.suggest_categorical('dense_bn', [True, False])
    batch_size = trial.suggest_int('batch_size', 32, 512, step=32)
    in_dropout = 0#trial.suggest_float('in_dropout', 0.0, 0.5, step=0.1)
    mid_dropout = 0#trial.suggest_float('mid_dropout', 0.0, 0.5, step=0.1)
    out_dropout = 0#trial.suggest_float('out_dropout', 0.0, 0.5, step=0.1)
    weight_decay = trial.suggest_float('weight_decay', 1e-7, 1, log=True)
    # rho = 0.08#trial.suggest_float('rho', 0.0, 0.1, step=0.01)


    train_loader, attack_loader, _, _, _, _ = load_ascad_data(device, batch_size=batch_size)

    if True:
        model = MLPModel(signal_length=signal_length, 
                         n_classes=n_classes, n_linear=n_linear, 
                         linear_size=linear_size, activation=activation, 
                         input_bn=input_bn, dense_bn=dense_bn).to(device)

        optimizer = Scoop(model.parameters(), lr=lr, 
                          betas=(beta1, beta2), 
                          weight_decay=weight_decay, estimator='low_variance',
                          hessian_iter=1)
        adam_optimizer = torch.optim.Adam(model.parameters(), lr=lr,
                                            betas=(beta1, beta2), weight_decay=weight_decay)

        path_model = 'scoop_vs_adam/ascadv1_fintuning_adam_' + str(trial.number) + '.pt'
        train_loss, val_loss, _ = train_model_classic(model, adam_optimizer, n_epochs, train_loader, attack_loader, verbose=True, path=path_model, device=device, MLP=False, finetuning=True, entropy=8)
        trial.set_user_attr('val_loss', val_loss)
        trial.set_user_attr('train_loss', train_loss)

        return np.mean(np.sort(np.array(val_loss))[:1])
    # except:
    #     return np.nan

In [3]:
storage_name = f"sqlite:///ascadv1_finetuning_scoop_comp.db"
tpe_sampler = optuna.samplers.TPESampler(seed=seed, n_startup_trials=10)
# random sampler
sampler = optuna.samplers.RandomSampler(seed=seed)
optuna.logging.set_verbosity(optuna.logging.INFO)
study = optuna.create_study(direction="minimize", storage=storage_name, study_name="adam", load_if_exists=True, sampler=sampler) 
study.optimize(objective, n_trials=167, n_jobs=1, show_progress_bar=True) 

[I 2025-01-09 17:16:39,666] A new study created in RDB with name: adam
  0%|          | 0/167 [00:00<?, ?it/s]

Best trial: 0. Best value: 8.002:   1%|          | 1/167 [00:05<15:10,  5.49s/it]

[I 2025-01-09 17:16:45,150] Trial 0 finished with value: 8.001999440400496 and parameters: {'lr': 0.0005611516415334506, 'beta1': 0.995, 'beta2': 0.973, 'activation': 'ReLU', 'n_linear': 8, 'linear_size': 256, 'input_bn': False, 'dense_bn': False, 'batch_size': 448, 'weight_decay': 3.064599841241146e-06}. Best is trial 0 with value: 8.001999440400496.


Best trial: 0. Best value: 8.002:   1%|          | 2/167 [00:14<20:13,  7.35s/it]

[I 2025-01-09 17:16:53,813] Trial 1 finished with value: 8.004759293568286 and parameters: {'lr': 0.0002310201887845295, 'beta1': 0.918, 'beta2': 0.93, 'activation': 'Tanh', 'n_linear': 2, 'linear_size': 256, 'input_bn': False, 'dense_bn': False, 'batch_size': 128, 'weight_decay': 0.0003977782830811189}. Best is trial 0 with value: 8.001999440400496.


...

[I 2025-01-09 17:47:43,137] Trial 164 finished with value: 8.003247661955038 and parameters: {'lr': 0.000274244305309583, 'beta1': 0.903, 'beta2': 0.91, 'activation': 'SeLU', 'n_linear': 4, 'linear_size': 256, 'input_bn': True, 'dense_bn': False, 'batch_size': 64, 'weight_decay': 2.6095282310615847e-05}. Best is trial 55 with value: 7.922266195690821.


Best trial: 55. Best value: 7.92227:  99%|█████████▉| 166/167 [31:03<00:17, 17.73s/it]

[I 2025-01-09 17:47:43,517] Trial 165 finished with value: 8.139888048171997 and parameters: {'lr': 0.0010347224631409363, 'beta1': 0.987, 'beta2': 0.9490000000000001, 'activation': 'SeLU', 'n_linear': 4, 'linear_size': 256, 'input_bn': True, 'dense_bn': True, 'batch_size': 320, 'weight_decay': 1.4642290706482846e-05}. Best is trial 55 with value: 7.922266195690821.


Best trial: 55. Best value: 7.92227: 100%|██████████| 167/167 [31:04<00:00, 11.16s/it]

[I 2025-01-09 17:47:43,852] Trial 166 finished with value: 8.554567074775695 and parameters: {'lr': 0.0044420226473872925, 'beta1': 0.995, 'beta2': 0.982, 'activation': 'SeLU', 'n_linear': 2, 'linear_size': 256, 'input_bn': False, 'dense_bn': False, 'batch_size': 256, 'weight_decay': 3.8315204893889454e-05}. Best is trial 55 with value: 7.922266195690821.


In [2]:
def objective(trial):
    n_classes = 256
    signal_length = 700
    lr = trial.suggest_float('lr', 1e-4, 1e-2, log=True)
    beta1 = trial.suggest_float('beta1', 0.9, 0.999, step=0.001)
    beta2 = trial.suggest_float('beta2', 0.9, 0.999, step=0.001)
    n_epochs = 60
    activation = trial.suggest_categorical('activation', ['ReLU', 'SeLU', 'ELU', 'Tanh'])
    n_linear = trial.suggest_int('n_linear', 2, 5)
    n_conv = trial.suggest_int('n_conv', 1, 5)
    conv_filter_type = trial.suggest_categorical('conv_filter_type', ['increasing', 'increasing_clipped', 'same'])
    kernel_size = trial.suggest_int('kernel_size', 5, 17, step=2)
    conv_filter_size = trial.suggest_int('conv_filter_size', 10,20, step=10)
    linear_size = trial.suggest_categorical('linear_size', [256])
    input_bn = trial.suggest_categorical('input_bn', [True, False])
    dense_bn = trial.suggest_categorical('dense_bn', [True, False])
    batch_size = trial.suggest_int('batch_size', 32, 512, step=32)
    pooling_size = trial.suggest_int('pooling_size', 2, 3)
    global_pool = trial.suggest_categorical('global_pool', [True, False])
    conv_bn = trial.suggest_categorical('conv_bn', [True, False])
    weight_decay = trial.suggest_float('weight_decay', 1e-7, 1, log=True)
    # rho = 0.08#trial.suggest_float('rho', 0.0, 0.1, step=0.01)


    train_loader, attack_loader, _, _, _, _ = load_ascad_data(device, batch_size=batch_size)

    try:
        model = CNNModel(n_classes, signal_length, n_linear=n_linear, 
                         n_conv=n_conv, conv_filter_type=conv_filter_type, 
                         kernel_size=kernel_size, conv_filter_size=conv_filter_size,
                         pooling_size=pooling_size, pooling_stride=None,
                         linear_size=linear_size, activation=activation, global_pool=global_pool,
                         input_bn=input_bn, dense_bn=dense_bn, conv_bn=conv_bn).to(device)

        optimizer = Scoop(model.parameters(), lr=lr, 
                          betas=(beta1, beta2), 
                          weight_decay=weight_decay, estimator='low_variance',
                          hessian_iter=1)
        adam_optimizer = torch.optim.Adam(model.parameters(), lr=lr,
                                            betas=(beta1, beta2), weight_decay=weight_decay)

        path_model = 'scoop_vs_adam/ascadv1_fintuning_scoop_cnn_' + str(trial.number) + '.pt'
        train_loss, val_loss, _ = train_model_classic(model, adam_optimizer, n_epochs, train_loader, attack_loader, verbose=True, path=path_model, device=device, MLP=False, finetuning=True, entropy=8)
        trial.set_user_attr('val_loss', val_loss)
        trial.set_user_attr('train_loss', train_loss)

        return np.mean(np.sort(np.array(val_loss))[:1])
    except:
        return np.nan

In [3]:
storage_name = f"sqlite:///ascadv1_finetuning_scoop_comp.db"
tpe_sampler = optuna.samplers.TPESampler(seed=seed, n_startup_trials=10)
# random sampler
sampler = optuna.samplers.RandomSampler(seed=seed)
optuna.logging.set_verbosity(optuna.logging.INFO)
study = optuna.create_study(direction="minimize", storage=storage_name, study_name="adam_cnn1", load_if_exists=True, sampler=sampler) 
study.optimize(objective, n_trials=98, n_jobs=1, show_progress_bar=True) 

[I 2025-01-10 07:43:54,684] A new study created in RDB with name: adam_cnn1
  0%|          | 0/98 [00:00<?, ?it/s]

Best trial: 0. Best value: 8.00171:   1%|          | 1/98 [01:03<1:43:00, 63.72s/it]

[I 2025-01-10 07:44:58,400] Trial 0 finished with value: 8.001707490285238 and parameters: {'lr': 0.0005611516415334506, 'beta1': 0.995, 'beta2': 0.973, 'activation': 'ReLU', 'n_linear': 5, 'n_conv': 4, 'conv_filter_type': 'same', 'kernel_size': 15, 'conv_filter_size': 10, 'linear_size': 256, 'input_bn': False, 'dense_bn': False, 'batch_size': 224, 'pooling_size': 2, 'global_pool': True, 'conv_bn': False, 'weight_decay': 0.0001557721770269301}. Best is trial 0 with value: 8.001707490285238.


Best trial: 0. Best value: 8.00171:   2%|▏         | 2/98 [01:08<46:33, 29.10s/it]  

[I 2025-01-10 07:45:03,273] Trial 1 finished with value: 15.287322680155436 and parameters: {'lr': 0.0037183641805732083, 'beta1': 0.919, 'beta2': 0.9510000000000001, 'activation': 'ELU', 'n_linear': 2, 'n_conv': 5, 'conv_filter_type': 'increasing', 'kernel_size': 5, 'conv_filter_size': 20, 'linear_size': 256, 'input_bn': True, 'dense_bn': True, 'batch_size': 480, 'pooling_size': 2, 'global_pool': True, 'conv_bn': False, 'weight_decay': 1.9678010532114947e-06}. Best is trial 0 with value: 8.001707490285238.


...

[I 2025-01-10 11:54:18,674] Trial 95 finished with value: 8.001585547129313 and parameters: {'lr': 0.0003554367658035554, 'beta1': 0.93, 'beta2': 0.978, 'activation': 'Tanh', 'n_linear': 5, 'n_conv': 3, 'conv_filter_type': 'increasing_clipped', 'kernel_size': 13, 'conv_filter_size': 20, 'linear_size': 256, 'input_bn': True, 'dense_bn': False, 'batch_size': 96, 'pooling_size': 3, 'global_pool': True, 'conv_bn': False, 'weight_decay': 0.0028728716403281537}. Best is trial 81 with value: 7.800514224228586.


Best trial: 81. Best value: 7.80051:  99%|█████████▉| 97/98 [4:11:50<04:18, 258.19s/it]

[I 2025-01-10 11:55:45,370] Trial 96 finished with value: 7.870500777856991 and parameters: {'lr': 0.00047996363429746457, 'beta1': 0.907, 'beta2': 0.9400000000000001, 'activation': 'SeLU', 'n_linear': 2, 'n_conv': 5, 'conv_filter_type': 'increasing_clipped', 'kernel_size': 11, 'conv_filter_size': 10, 'linear_size': 256, 'input_bn': False, 'dense_bn': True, 'batch_size': 32, 'pooling_size': 2, 'global_pool': True, 'conv_bn': True, 'weight_decay': 1.0585549414012343e-05}. Best is trial 81 with value: 7.800514224228586.


Best trial: 81. Best value: 7.80051: 100%|██████████| 98/98 [4:11:56<00:00, 154.25s/it]

[I 2025-01-10 11:55:50,754] Trial 97 finished with value: 8.007002706113068 and parameters: {'lr': 0.0063119909283947005, 'beta1': 0.911, 'beta2': 0.995, 'activation': 'SeLU', 'n_linear': 5, 'n_conv': 2, 'conv_filter_type': 'increasing', 'kernel_size': 9, 'conv_filter_size': 20, 'linear_size': 256, 'input_bn': False, 'dense_bn': False, 'batch_size': 448, 'pooling_size': 2, 'global_pool': True, 'conv_bn': True, 'weight_decay': 2.4953751842497307e-05}. Best is trial 81 with value: 7.800514224228586.


In [ ]:
def func(c):
    return (np.exp(2*c)+np.exp(2))/(np.exp(2*c+1)+np.exp(2))